In [ ]:
import csv
import gradio as gr
from datetime import datetime

def save_data(dob, name, mobile, mulank, bhagyank, name_number, sum_mobile, lucky_numbers, result):
    """Saves user details and result to a CSV file."""
    with open("user_data.csv", mode="a", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        if file.tell() == 0:
            writer.writerow(["Timestamp", "DOB", "Name", "Mobile", "Mulank", "Bhagyank", "Name Number", "Sum of Mobile", "Lucky Numbers", "Result"])
        writer.writerow([datetime.now(), dob, name, mobile, mulank, bhagyank, name_number, sum_mobile, ', '.join(map(str, lucky_numbers)), result])

def get_single_digit(num):
    while num > 9:
        num = sum(int(d) for d in str(num))
    return num

def calculate_lucky_number(dob, name, mobile):
    if not dob or not name or not mobile:
        return "❌ DOB, Name, and Mobile Number are required!"
    if len(mobile) != 10 or not mobile.isdigit():
        return "❌ Invalid Mobile Number! Enter a 10-digit number."
    
    try:
        datetime.strptime(dob, "%d-%m-%Y")
    except ValueError:
        return "❌ Invalid DOB format! Use DD-MM-YYYY."
    
    letter_values = {
        'a': 1, 'i': 1, 'j': 1, 'q': 1, 'y': 1,
        'b': 2, 'k': 2, 'r': 2,
        'c': 3, 'g': 3, 'l': 3, 's': 3,
        'd': 4, 'm': 4, 't': 4,
        'e': 5, 'h': 5, 'n': 5, 'x': 5,
        'u': 6, 'v': 6, 'w': 6,
        'o': 7, 'z': 7,
        'f': 8, 'p': 8
    }
    
    name_num = sum(letter_values[char] for char in name.lower() if char in letter_values)
    
    mulank = get_single_digit(sum(int(d) for d in dob.split('-')[0]))
    bhagyank = get_single_digit(sum(int(d) for d in dob.replace('-', '')))
    name_number = get_single_digit(name_num)
    sum_mobile = get_single_digit(sum(int(d) for d in mobile))
    
    friendly_numbers = {
        1: [1,2,3,5,6,9], 2: [1,2,3,5], 3: [1,2,3,5,7], 4: [1,7,5,6,4,8], 
        5: [1,2,3,5,6], 6: [1,7,5,6], 7: [1,3,5,4,6], 8: [5,3,6,7,4,8], 9: [1,5,3]
    }
    
    lucky_numbers = list(set(friendly_numbers.get(mulank, [])) & 
                         set(friendly_numbers.get(bhagyank, [])) & 
                         set(friendly_numbers.get(name_number, [])))
    
    is_lucky = sum_mobile in lucky_numbers
    result = f"🎉 Congratulations! {mobile} is your Lucky Mobile Number!" if is_lucky else "❌ This is not a lucky number. Try again!"
    
    save_data(dob, name, mobile, mulank, bhagyank, name_number, sum_mobile, lucky_numbers, result)
    return result

def lucky_number_interface(dob, name, mobile):
    return calculate_lucky_number(dob, name, mobile)

iface = gr.Interface(
    fn=lucky_number_interface,
    inputs=["text", "text", "text"],
    outputs="text",
    title="Lucky Number Checker",
    description="Enter your Date of Birth (DD-MM-YYYY), Name, and Mobile Number to check if it's lucky!"
)

if __name__ == "__main__":
    iface.launch(debug=True,share = True)
